# Prompt Engineering

In this notebook we will go through the fundamentals of prompt engineering.

### Prerequisties

First, you'll need to get an [OpenAI API key](https://openai.com/docs/quickstart/get-api-key) and a [Pinecone API key](https://www.pinecone.io/docs/python/get-started#step-1-get-an-api-key).

Next, you'll need to install some Python libraries:

- [langchain](https://pypi.org/project/langchain/): A library for working with blockchain-based languages.
- [openai](https://pypi.org/project/openai/): The OpenAI Python client library for accessing the OpenAI API.
- [pinecone-client](https://pypi.org/project/pinecone-client/): Python client library for Pinecone, a vector database service.


In [1]:
import os

from langchain.schema import(
    SystemMessage,
    HumanMessage,
    AIMessage
)

from langchain_openai import ChatOpenAI

In [2]:
openapi_key = os.getenv('OPENAI_API_KEY')

In [8]:
chat = ChatOpenAI(
    openai_api_key = openapi_key,
    model = 'gpt-3.5-turbo'
)

Here we will ask it a very simple quesiton, to explain what prompt enginnering is.

In [9]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hi AI, how you doing?"),
    AIMessage(content = "I'm doing great. How can I help you?"),
    HumanMessage(content="What is Prompt engineering")
]

In [10]:
response = chat(messages)
response

/home/lemi/Documents/week7/Precision-RAG/week7_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='Prompt engineering is a process that involves designing and implementing prompts in a structured and effective way to guide users through a specific task or interaction. Prompts can be in the form of messages, cues, instructions, or feedback that help users understand what actions to take next. Effective prompt engineering can improve user experience, increase usability, and reduce errors in user interactions with a system or interface.', response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 46, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-42cffb38-4355-49ca-b0d6-f55c320284d3-0', usage_metadata={'input_tokens': 46, 'output_tokens': 76, 'total_tokens': 122})

We can add this `response` back to the messages since it is just another `AIMessage` 

In [14]:
messages.append(response)

prompt = HumanMessage(content="What is RAG")

messages.append(prompt)

response = chat(messages)

print(response.content)

RAG stands for Red, Amber, and Green, which are colors commonly used in project management to indicate the status of a project or task. 

- Red typically signifies that a project or task is behind schedule, over budget, or facing significant issues that require immediate attention.
- Amber or Yellow indicates that there are some concerns or risks that need to be addressed, but the project or task is still on track overall.
- Green signifies that a project or task is on schedule, within budget, and meeting expectations.

RAG status is commonly used to provide a quick visual representation of the health and progress of a project, allowing stakeholders to easily identify areas that may require intervention or support.


Here since we didn't specify the content, it gave us unexpected result

In [16]:
prompt = HumanMessage(content="What is RAG, related to LLM's (Retrieval Augmented Generation)?")

messages.append(prompt)

response = chat(messages)

print(response.content)

In the context of LLMs (Large Language Models) like GPT-3 (Generative Pre-trained Transformer 3), RAG stands for Retrieval-Augmented Generation. RAG is a model that combines elements of retrieval-based and generative-based approaches to natural language processing. It involves first retrieving relevant information from a knowledge source, such as a database or a set of documents, and then using that information to generate text or responses. This approach allows the model to incorporate external knowledge into its generated outputs, making them more accurate and informative.


## Limitations of Large Language Models: Exploring Context-Dependence

While large language models (LLMs) like GPT-3.5 are impressive in their ability to generate text, translate languages, and answer questions, they have limitations, particularly when it comes to information that's:

- **Real-time or Location-Specific:** LLMs are typically trained on static datasets and lack access to real-time information. Questions like "What's the current stock price of Tesla?" or "What's that loud noise outside?" cannot be definitively answered by the model since it doesn't have access to live market data or your physical environment's sounds.

- **Highly Subjective or Personal:**  "What is the best career path for me?" is a question that depends heavily on individual factors. LLMs might offer generic advice, but they can't provide personalized recommendations.


In [21]:
prompt = HumanMessage(content="What's the current stock price of Tesla?")

messages.append(prompt)

response = chat(messages)

print(response.content)

I'm unable to provide real-time stock prices. You can easily check the current stock price of Tesla by searching for it on a financial news website, a stock market app, or by using a search engine like Google.


In [22]:
prompt = HumanMessage(content="What is the best career path for me?")

messages.append(prompt)

response = chat(messages)

print(response.content)

I'm here to provide guidance and suggestions. To help determine the best career path for you, it would be helpful to know more about your interests, skills, values, and goals. Reflecting on what you enjoy doing, what you are good at, and what matters to you can be a good starting point. Additionally, seeking advice from career counselors, taking career assessments, exploring different industries, and gaining experience through internships or volunteering can also help you discover a suitable career path. Feel free to share more about yourself so I can offer more tailored suggestions.


**The Power of Context:**

The key to unlocking a LLM's full potential lies in providing it with relevant context. When a question is specific and accompanied by related content, the LLM can leverage its knowledge and capabilities to craft a more informative response.

**Example: Leveraging Context for Improved Answers**

Let's consider a scenario where you're researching a historical event:

In [24]:
context = """
The Battle of Marathon was a pivotal conflict in the Greco-Persian Wars, fought in 490 BC near the town of Marathon in Attica, Greece. A numerically outnumbered Athenian army decisively defeated a larger Persian force.
"""

query = "What were the key strategies employed by the Athenians during the Battle of Marathon?"

prompt = f"""Using the contexts below, answer the query.

Contexts:
{context}

Query:
{query}
"""

# Question with context
prompt = HumanMessage(content=prompt)

response = chat(messages)

print(response.content)

Determining the best career path for you depends on your interests, skills, values, and goals. It's important to consider what you enjoy doing, what you are good at, and what kind of impact you want to make in your career. You may want to explore different industries, job roles, and opportunities to find a career path that aligns with your strengths and aspirations. It can also be helpful to seek guidance from career counselors, mentors, or professionals in fields you are interested in. Remember that career paths are not set in stone and it's okay to explore different options to find the right fit for you.


# Understanding Pinecone

While large language models (LLMs) like GPT-3.5 are adept at processing text, their effectiveness in answering questions heavily relies on providing them with relevant context. In real-world scenarios, this context rarely appears as a neatly formatted string within the code. It often resides in various formats like documents, CSV files, or Excel spreadsheets.

This is where the power of embedding models and vector databases comes into play. They act as a bridge between the raw data in your files and the LLM, enabling it to understand and utilize the information effectively.

### The Role of Embeddings and Vector Databases:

#### Data Preprocessing:

    The first step involves processing your documents, CSV, or Excel files to extract meaningful textual content. This might involve techniques like text cleaning, normalization, and tokenization (breaking down the text into individual words or phrases).

#### Embedding Model:

    Once you have the processed text, an embedding model takes over. This model, often trained on massive datasets, maps each piece of text (e.g., a word, sentence, or document) into a numerical vector representation. These vectors capture the semantic relationships between words and concepts, allowing the model to understand the meaning and context of the text.

    Popular choices for embedding models include:

    Universal Sentence Encoder (USE) from Google AI
    Sentence Transformers from Hugging Face (https://huggingface.co/sentence-transformers)

#### Vector Database:

    The generated numerical vectors, representing the meaning of your text data, are then stored in a vector database. This database is optimized for performing efficient similarity searches based on vector distances.

    Examples of vector databases for chatbot applications include:

    Pinecone (https://www.pinecone.io/)
    Faiss (Facebook AI Similarity Search) (https://github.com/facebookresearch/faiss)


In [33]:
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.openai import OpenAIEmbeddings

In [55]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]

In [53]:
import openai

In [54]:
embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
result = embed_model.embed_documents(sentences)

print(result)
print(len(result[0]))

[[0.0015969427303684175, 0.00599949463516727, -0.015092674203844027, -0.00854834051155039, -0.011523037813742506, 0.0156437768707638, -0.021543070047381173, -0.008673591286999892, -0.010984461435086995, -0.022444873395443477, 0.021280044070863, 0.01764778369002055, 0.02200649800967659, -0.0025832896766502873, 0.0007675501760377505, -0.012199390324789233, 0.02600198535992583, -0.008567128081301688, 0.013727444756131406, -0.01230585259916489, -0.013464418779613234, 0.023208900611114563, -0.00526364877412313, -0.0052699109869327136, 0.00803481391545576, 0.02105459509649252, 0.010389521415855044, 0.000869707528352774, 0.002782124712241312, -0.0057897002614982, 0.015869227707779373, -0.0021699633749523686, -0.0362725210071086, -0.0012572010380579764, -0.01094062315145227, -0.014178346430162556, 0.00206663174132023, -0.006581909297457507, 0.024286055231070677, -0.046593154356768036, 0.011429099964986016, 0.021668321754153223, -0.025701384243905064, -0.005874244325379041, 0.002342182609118842

In [61]:
client = Pinecone(api_key="pinecone_api_key")#, environment="example")

index_name = "text-example-index"

pc = Pinecone(pinecone_api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

#index = pc.Index(index_name)

# Create an index in Pinecone (adjust parameters as needed)
#index = client.create_index("my_embeddings_index", dimension=768)  # Adjust dimension based on model output

# Store the embeddings in the index (replace with your actual embedding data)
namespace = "example_space" 
docsearch = PineconeVectorStore.from_texts(
    texts=sentences,
    index_name=index_name,
    embedding = result,
    namespace=namespace
)

AttributeError: 'list' object has no attribute 'embed_documents'

In [62]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]


embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
result = embed_model.embed_documents(sentences)

print(result)
print(len(result[0]))

client = Pinecone(api_key="pinecone_api_key")#, environment="example")

index_name = "text-example-index"

pc = Pinecone(pinecone_api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Option 1: Use PineconeVectorStore.from_texts (preferred)
docsearch = PineconeVectorStore.from_texts(
    texts=sentences,
    embedding=result.reshape(-1, result.shape[1]).tolist(),  # Reshape and convert to list
    index_name=index_name,
    namespace="example_space"
)

# Option 2: Iterate and add texts with individual embeddings
# for sentence, embedding in zip(sentences, result):
#   docsearch.add_texts(texts=[sentence], embedding=embedding.tolist())


[[0.001602984597358992, 0.005992407376668513, -0.015053027138167847, -0.008572210460203617, -0.011559021527649125, 0.015566482962838945, -0.02152758253453419, -0.008634827001472712, -0.010964164385592716, -0.02240421411230153, 0.02132720848488601, 0.01764535325056, 0.022053561853723622, -0.0025610183074188876, 0.0007146117778194333, -0.012216496142297193, 0.025973360689930246, -0.008565949178605732, 0.013662938804406844, -0.012341729224835386, -0.013462565686081226, 0.02319318402240824, -0.005284839715269681, -0.005212830785942477, 0.008102586400685398, 0.021051695330772963, 0.010337998041579194, 0.0008883728195395586, 0.0027880035023500004, -0.005770118375766455, 0.015791902138878662, -0.0021070481503873023, -0.036292574886715785, -0.0012233715830844581, -0.010895285631403172, -0.01422648860715127, 0.0020913940150700285, -0.006562217995349541, 0.024320281765251965, -0.04658674023181954, 0.011433788445110933, 0.02157767464996239, -0.025672800546780537, -0.005845258411553882, 0.00233559

AttributeError: 'list' object has no attribute 'reshape'

In [2]:
from langchain_pinecone import PineconeVectorStore
import os

pinecone_api_key = os.getenv('PINECONE_API_KEY')
openapi_key = os.getenv('OPENAI_API_KEY')

In [5]:
import openai

In [7]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]

model_name = "text-embedding-ada-002"  # Example model - adjust as needed

# Generate embeddings using the model
def embed_sentences(sentences, model_name):
  embeddings = []
  for sentence in sentences:
    response = openai.Embedding.create(
        engine=model_name,
        input=sentence
    )
    embedding = response["data"][0]["embedding"]
    embeddings.append(embedding)
  return embeddings

embeddings = embed_sentences(sentences, model_name)

print("Example Embeddings:")
print(embeddings[:3])

Example Embeddings:
[[0.001636281842365861, 0.006003871560096741, -0.015042529441416264, -0.00854120310395956, -0.011494644917547703, 0.015593171119689941, -0.021487539634108543, -0.008628805167973042, -0.010925231501460075, -0.022401105612516403, 0.021299822255969048, 0.01765807718038559, 0.02205069549381733, -0.0026218052953481674, 0.0007336674607358873, -0.012182947248220444, 0.026005305349826813, -0.008572489954531193, 0.013653410598635674, -0.012295578606426716, -0.013465692289173603, 0.023151978850364685, -0.005249867681413889, -0.005259253550320864, 0.008103192783892155, 0.02106204442679882, 0.010380847379565239, 0.0008814960019662976, 0.002773544518277049, -0.0058098952285945415, 0.015855977311730385, -0.002180666197091341, -0.03624223545193672, -0.0011740244226530194, -0.010981546714901924, -0.014141479507088661, 0.002107142936438322, -0.006570156197994947, 0.024303320795297623, -0.04660430923104286, 0.011438329704105854, 0.021650230512022972, -0.025654897093772888, -0.0058317

In [ ]:

pinecone_api_key = os.getenv('PINECONE_API_KEY')
openapi_key = os.getenv('OPENAI_API_KEY')

In [12]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.openai import OpenAIEmbeddings

In [13]:

sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]


embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
result = embed_model.embed_documents(sentences)

print(result)
print(len(result[0]))

client = Pinecone(api_key="pinecone_api_key")#, environment="example")

index_name = "text-example-index"

pc = Pinecone(pinecone_api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Option 1: Use PineconeVectorStore.from_texts (preferred)
docsearch = PineconeVectorStore.from_texts(
    texts=sentences,
    embedding=result.reshape(-1, result.shape[1]).tolist(),  # Reshape and convert to list
    index_name=index_name,
    namespace="example_space"
)

/home/lemi/Documents/week7/Precision-RAG/week7_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[[0.001602984597358992, 0.005992407376668513, -0.015053027138167847, -0.008572210460203617, -0.011559021527649125, 0.015566482962838945, -0.02152758253453419, -0.008634827001472712, -0.010964164385592716, -0.02240421411230153, 0.02132720848488601, 0.01764535325056, 0.022053561853723622, -0.0025610183074188876, 0.0007146117778194333, -0.012216496142297193, 0.025973360689930246, -0.008565949178605732, 0.013662938804406844, -0.012341729224835386, -0.013462565686081226, 0.02319318402240824, -0.005284839715269681, -0.005212830785942477, 0.008102586400685398, 0.021051695330772963, 0.010337998041579194, 0.0008883728195395586, 0.0027880035023500004, -0.005770118375766455, 0.015791902138878662, -0.0021070481503873023, -0.036292574886715785, -0.0012233715830844581, -0.010895285631403172, -0.01422648860715127, 0.0020913940150700285, -0.006562217995349541, 0.024320281765251965, -0.04658674023181954, 0.011433788445110933, 0.02157767464996239, -0.025672800546780537, -0.005845258411553882, 0.00233559

AttributeError: 'list' object has no attribute 'reshape'

In [1]:
import numpy as np
from openai.embeddings_utils import OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from pinecone.vectorstores import PineconeVectorStore
import os

sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]

pinecone_api_key = os.getenv('PINECONE_API_KEY')
openapi_key = os.getenv('OPENAI_API_KEY')

#pc = Pinecone(pinecone_api_key)

# Embed the sentences
embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
result = embed_model.embed_documents(sentences)

# Convert the result to a NumPy array and reshape
embeddings = np.array(result).reshape(-1, len(result[0])).tolist()

# Initialize Pinecone
client = Pinecone(api_key=pinecone_api_key)

index_name = "text-example-index"

if index_name not in client.list_indexes().names():
    client.create_index(
        name=index_name,
        dimension=128,  # Adjust the dimension as needed
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Use PineconeVectorStore.from_texts
docsearch = PineconeVectorStore.from_texts(
    texts=sentences,
    embedding=embeddings,  # Use the reshaped embeddings list
    index_name=index_name,
    namespace="example_space"
)


ModuleNotFoundError: No module named 'scipy'

In [10]:
client = Pinecone(api_key="pinecone_api_key")#, environment="example")

index_name = "text-example-index"

pc = Pinecone(pinecone_api_key)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

#index = pc.Index(index_name)

# Create an index in Pinecone (adjust parameters as needed)
#index = client.create_index("my_embeddings_index", dimension=768)  # Adjust dimension based on model output

# Store the embeddings in the index (replace with your actual embedding data)
namespace = "example_space" 
docsearch = PineconeVectorStore.from_documents(
    documents=sentences,
    index_name=index_name,
    embedding = result,
    namespace=namespace
)

NameError: name 'result' is not defined

In [28]:
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openapi_key = os.getenv('OPENAI_API_KEY')


In [29]:
pc = Pinecone(pinecone_api_key)

In [14]:
index_name = "text-example-index"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec = ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [31]:
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "Climate change is a pressing issue that requires immediate action.",
    "The capital of France is Paris, a beautiful city known for its Eiffel Tower."
]

In [12]:
# Step 1: Install Pinecone
# pip install pinecone-client

# Step 2: Import Pinecone and Initialize Client
import pinecone
import numpy as np

# Initialize the Pinecone client
#pinecone.init(api_key=pinecone_api_key)
pinecone = Pinecone(api_key=pinecone_api_key)


# Step 3: Create an Index
index_name = "text-example-index"
#pinecone.create_index(name=index_name, dimension=128, metric='cosine')
pinecone.create_index(name=index_name, spec={"dimension": 128, "metric": 'cosine'})

# Step 4: Connect to the Index
index = pinecone.Index(index_name)

# Example text documents
documents = [
    {"id": "doc1", "text": "The quick brown fox jumps over the lazy dog."},
    {"id": "doc2", "text": "A fast brown fox leaps over a sleepy dog."},
    {"id": "doc3", "text": "Lorem ipsum dolor sit amet, consectetur adipiscing elit."},
    {"id": "doc4", "text": "A journey of a thousand miles begins with a single step."},
]

# Precomputed embeddings (using random vectors here for simplicity)
embeddings = [
    {"id": "doc1", "values": np.random.rand(128).tolist()},
    {"id": "doc2", "values": np.random.rand(128).tolist()},
    {"id": "doc3", "values": np.random.rand(128).tolist()},
    {"id": "doc4", "values": np.random.rand(128).tolist()},
]

# Step 5: Insert Vectors
index.upsert(embeddings)

# Step 6: Query Vectors
# Assume we have a query text and its embedding
query_text = "A quick brown fox jumps over the dog."
query_embedding = np.random.rand(128).tolist()  # Using a random vector for simplicity

# Query the index
results = index.query(queries=[query_embedding], top_k=2)

# Display the results
print("Query Results for:", query_text)
for match in results['matches']:
    doc_id = match['id']
    score = match['score']
    matched_doc = next(doc for doc in documents if doc['id'] == doc_id)
    print(f"ID: {doc_id}, Score: {score}, Text: {matched_doc['text']}")

# Clean up by deleting the index
#pinecone.delete_index(index_name)


TypeError: create_index() missing 1 required positional argument: 'dimension'

In [3]:
#from langchain.text_splitter import SentenceSplitter
#from langchain.document_loaders import PDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from PyPDF2 import PdfReader


In [5]:
filename = '../prompts/10 Academy Cohort B - Weekly Challenge_ Week - 7.pdf'
content = ""
with open(filename, 'rb') as f:
    reader = PdfReader(f)
    for page in reader.pages:
        content += page.extract_text()

content


'evaluate\n10\nAcademy\nCohort\nB\nWeekly\nChallenge:\nWeek\n7\nPrecision\nRAG:\nPrompt\nTuning\nFor\nBuilding\nEnterprise\nGrade\nRAG\nSystems\nBusiness\nobjective\nPromptlyTech\nis\nan\ninnovative\ne-business\nspecializing\nin\nproviding\nAI-driven\nsolutions\nfor\noptimizing\nthe\nuse\nof\nLanguage\nModels\n(LLMs)\nin\nvarious\nindustries.\nThe\ncompany\naims\nto\nrevolutionize\nhow\nbusinesses\ninteract\nwith\nLLMs,\nmaking\nthe\ntechnology\nmore\naccessible,\neﬃcient,\nand\neﬀective.\nBy\naddressing\nthe\nchallenges\nof\nprompt\nengineering,\nthe\ncompany\nplays\na\npivotal\nrole\nin\nenhancing\ndecision-making,\noperational\neﬃciency,\nand\ncustomer\nexperience\nacross\nvarious\nindustries.\nPromptlyTech\'s\nsolutions\nare\ndesigned\nto\ncater\nto\nthe\nevolving\nneeds\nof\na\ndigitally-driven\nbusiness\nlandscape,\nwhere\nspeed\nand\naccuracy\nare\nkey\nto\nstaying\ncompetitive.\nThe\ncompany\nfocuses\non\nkey\nservices:\nAutomatic\nPrompt\nGeneration,\nAutomatic\nEvaluation\nDa

In [8]:
import os
pinecone_api_key = os.getenv('PINECONE_API_KEY')
openapi_key = os.getenv('OPENAI_API_KEY')


In [9]:
pinecone_client = Pinecone(api_key=pinecone_api_key, environment="example")

/home/lemi/Documents/week7/Precision-RAG/week7_env/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 0.3.0. An updated version of the class exists in the langchain-pinecone package and should be used instead. To use it run `pip install -U langchain-pinecone` and import as `from langchain_pinecone import Pinecone`.
  warn_deprecated(


TypeError: __init__() got an unexpected keyword argument 'api_key'